In [25]:
import polars as pl

### Funciones

In [ ]:
def contar_valores_repetidos(df: pl.DataFrame, columna: str):
    # Número de valores únicos
    num_unicos = df[columna].n_unique()
    print(f"Número de valores únicos en '{columna}': {num_unicos}")

    # Conteo de valores repetidos
    conteo = (
        df.group_by(columna)
          .count()
          .sort("count", descending=True)
    )
    print(f"Conteo de valores repetidos en '{columna}':")
    print(conteo)
    return conteo


def eliminar_filas_invalidas(df: pl.DataFrame) -> pl.DataFrame:
    # Asegurar que las columnas de fecha estén en formato de fecha
    df = df.with_columns([
        pl.col('fecha_convocatoria').cast(pl.Date),
        pl.col('fecha_buenapro').cast(pl.Date),
        pl.col('fecha_consentimiento_bp').cast(pl.Date)
    ])

    # Condiciones de validez
    condiciones_validas = (
        (pl.col('fecha_convocatoria') <= pl.col('fecha_buenapro')) &
        (pl.col('fecha_convocatoria') <= pl.col('fecha_consentimiento_bp')) &
        (pl.col('fecha_buenapro') <= pl.col('fecha_consentimiento_bp'))
    )

    # Contar antes de filtrar
    filas_antes = df.height

    # Aplicar filtro
    df_filtrado = df.filter(condiciones_validas)

    # Contar después de filtrar
    filas_despues = df_filtrado.height
    eliminadas = filas_antes - filas_despues

    print(f"✅ Filas eliminadas por inconsistencia de fechas: {eliminadas}")
    return df_filtrado

def agregar_diferencias_de_fecha(df: pl.DataFrame) -> pl.DataFrame:
    # Asegurar formato fecha
    df = df.with_columns([
        pl.col('fecha_convocatoria').cast(pl.Date),
        pl.col('fecha_buenapro').cast(pl.Date),
        pl.col('fecha_consentimiento_bp').cast(pl.Date)
    ])

    # Crear columnas de diferencias en días
    df = df.with_columns([
        (pl.col('fecha_buenapro') - pl.col('fecha_convocatoria')).dt.days().alias('f_convo_buenapro'),
        (pl.col('fecha_consentimiento_bp') - pl.col('fecha_buenapro')).dt.days().alias('f_buenapro_consentimiento')
    ])

    return df

In [ ]:
import polars as pl
import re

# Ruta del archivo
ruta_archivo1 = "Data/CONOSCE_ADJUDICACIONES2018_0.xlsx"
ruta_archivo2 = "Data/CONOSCE_ADJUDICACIONES2019_0.xlsx"
ruta_archivo3 = "Data/CONOSCE_ADJUDICACIONES2020_0.xlsx"
ruta_archivo4 = "Data/CONOSCE_ADJUDICACIONES2021_0.xlsx"
ruta_archivo5 = "Data/CONOSCE_ADJUDICACIONES2022_0.xlsx"
ruta_archivo6 = "Data/CONOSCE_ADJUDICACIONES2023_0.xlsx"
ruta_archivo7 = "Data/CONOSCE_ADJUDICACIONES2024_0.xlsx"

# Leer el archivo Excel
data = pl.read_excel(ruta_archivo)

# Extraer el año del nombre del archivo usando regex
match = re.search(r"(\d{4})", ruta_archivo)
year = int(match.group(1)) if match else None

# Agregar columna 'id' y nueva columna 'year'
data = data.with_columns([
    pl.concat_str(["descripcion_item", "entidad", "entidad_departamento"], separator=" ").alias("id"),
    pl.lit(year).alias("year")
])

# Definiendo el umbral
df_filtrado = data.filter(pl.col("monto_adjudicado_item") > 5_000_000)

# Contar valores únicos
conteo_ids = contar_valores_repetidos(df_filtrado, "id")
#guardando en un archivo CSV
conteo_ids.write_csv("valores_repetidos.csv")

Número de valores únicos en 'id': 1267
Conteo de valores repetidos en 'id':
shape: (1_267, 2)
┌─────────────────────────────────┬───────┐
│ id                              ┆ count │
│ ---                             ┆ ---   │
│ str                             ┆ u32   │
╞═════════════════════════════════╪═══════╡
│ EJECUCION DE LA OBRA CREACION … ┆ 2     │
│ UNIFORME DE DAMAS MINISTERIO P… ┆ 2     │
│ EJECUCION DEL SALDO DE OBRA : … ┆ 2     │
│ EJECUCION DE LA OBRA MEJORAMIE… ┆ 2     │
│ EJECUCI¿N DE LA OBRA: CONSTRUC… ┆ 2     │
│ …                               ┆ …     │
│ MEJORAMIENTO Y REHABILITACI¿N … ┆ 1     │
│ MEJORAMIENTO DE LOS SERVICIOS … ┆ 1     │
│ CONTRATACI¿N DE LA EJECUCI¿N D… ┆ 1     │
│ SERVICIO DE SEGURIDAD Y VIGILA… ┆ 1     │
│ MEJORAMIENTO Y AMPLIACION DEL … ┆ 1     │
└─────────────────────────────────┴───────┘


C:\Users\abrah\AppData\Local\Temp\ipykernel_31112\1363213197.py:11: DeprecationWarning: `GroupBy.count` was renamed; use `GroupBy.len` instead
  .count()


# FILTRADO por las columnas que vamos a usar

In [28]:
df_filtrado_more = df_filtrado.select([
    'year',
    'codigoconvocatoria',
    'descripcion_item',
    'estado_item',
    'monto_referencial_item',
    'monto_adjudicado_item',
    'moneda',
    'fecha_convocatoria',
    'fecha_buenapro',
    'fecha_consentimiento_bp',
    'tipoprocesoseleccion',
    'objetocontractual',
    'entidad_departamento',
    'entidad'
])

In [29]:
df_filtrado_more

year,codigoconvocatoria,descripcion_item,estado_item,monto_referencial_item,monto_adjudicado_item,moneda,fecha_convocatoria,fecha_buenapro,fecha_consentimiento_bp,tipoprocesoseleccion,objetocontractual,entidad_departamento,entidad
i32,i64,str,str,f64,f64,str,date,date,date,str,str,str,str
2018,494521,"""EJECUCI¿N DEL PROYECTO: ¿MEJOR…","""Contratado""",7.9893e6,7.9893e6,"""Soles""",2018-11-19,2018-12-13,2018-12-14,"""Procedimiento Especial de Cont…","""Obra""","""LA LIBERTAD""","""MUNICIPALIDAD PROVINCIAL DE OT…"
2018,466513,"""CONTRATACI¿N DE EJECUCI¿N DE O…","""Contratado""",6.1357e6,6.1357e6,"""Soles""",2018-08-10,2018-11-17,2018-12-03,"""Licitación Pública""","""Obra""","""SAN MARTIN""","""GOBIERNO REGIONAL DE SAN MARTI…"
2018,500600,"""EJECUCI¿N DE OBRA MEJORAMIENTO…","""Contratado""",5896191.2,5896191.2,"""Soles""",2018-10-23,2018-12-20,2018-12-20,"""Licitación Pública""","""Obra""","""UCAYALI""","""GOBIERNO REGIONAL DE UCAYALI S…"
2018,460199,"""ADQUISICION DEL SUMINISTRO DE …","""Contratado""",9.1256e6,9098223.2,"""Soles""",2018-07-17,2018-08-17,2018-09-03,"""Licitación Pública""","""Bien""","""LIMA""","""HOSPITAL CAYETANO HEREDIA"""
2018,504515,"""CONTRATACI¿N DEL SERVICIO DE S…","""Contratado""",5.6953e6,5.6913e6,"""Soles""",2018-12-27,2019-01-15,2019-01-23,"""Adjudicación Simplificada""","""Servicio""","""CUSCO""","""UNIVERSIDAD NACIONAL DE SAN AN…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018,467305,"""EJECUCION DE LA OBRA: MEJORAMI…","""Contratado""",5.2887e6,5.2887e6,"""Soles""",2018-07-17,2018-09-26,2018-09-27,"""Licitación Pública""","""Obra""","""HUANUCO""","""MUNICIPALIDAD DISTRITAL DE YAC…"
2018,493464,"""MEJORAMIENTO DE TROCHA CARROZA…","""Contratado""",1.3975e7,1.2577e7,"""Soles""",2018-11-15,2018-11-29,2018-12-04,"""Procedimiento Especial de Cont…","""Obra""","""LA LIBERTAD""","""MUNICIPALIDAD DISTRITAL DE CAC…"
2018,494491,"""EJECUCION DE OBRA: MEJORAMIENT…","""Contratado""",1.6903e7,1.6903e7,"""Soles""",2018-10-05,2018-12-04,2018-12-06,"""Licitación Pública""","""Obra""","""UCAYALI""","""GOBIERNO REGIONAL DE UCAYALI S…"


## Stop

In [30]:
df_limpio = eliminar_filas_invalidas(df_filtrado_more)

✅ Filas eliminadas por inconsistencia de fechas: 58


In [31]:
import polars as pl

def agregar_diferencias_de_fecha(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns([
        (pl.col('fecha_buenapro') - pl.col('fecha_convocatoria')).alias('f_convo_buenapro'),
        (pl.col('fecha_consentimiento_bp') - pl.col('fecha_buenapro')).alias('f_buenapro_consentimiento')
    ])

    return df


In [ ]:
df = agregar_diferencias_de_fecha(df_limpio)


In [33]:
df

year,codigoconvocatoria,descripcion_item,estado_item,monto_referencial_item,monto_adjudicado_item,moneda,fecha_convocatoria,fecha_buenapro,fecha_consentimiento_bp,tipoprocesoseleccion,objetocontractual,entidad_departamento,entidad,f_convo_buenapro,f_buenapro_consentimiento
i32,i64,str,str,f64,f64,str,date,date,date,str,str,str,str,duration[ms],duration[ms]
2018,494521,"""EJECUCI¿N DEL PROYECTO: ¿MEJOR…","""Contratado""",7.9893e6,7.9893e6,"""Soles""",2018-11-19,2018-12-13,2018-12-14,"""Procedimiento Especial de Cont…","""Obra""","""LA LIBERTAD""","""MUNICIPALIDAD PROVINCIAL DE OT…",24d,1d
2018,466513,"""CONTRATACI¿N DE EJECUCI¿N DE O…","""Contratado""",6.1357e6,6.1357e6,"""Soles""",2018-08-10,2018-11-17,2018-12-03,"""Licitación Pública""","""Obra""","""SAN MARTIN""","""GOBIERNO REGIONAL DE SAN MARTI…",99d,16d
2018,500600,"""EJECUCI¿N DE OBRA MEJORAMIENTO…","""Contratado""",5896191.2,5896191.2,"""Soles""",2018-10-23,2018-12-20,2018-12-20,"""Licitación Pública""","""Obra""","""UCAYALI""","""GOBIERNO REGIONAL DE UCAYALI S…",58d,0ms
2018,460199,"""ADQUISICION DEL SUMINISTRO DE …","""Contratado""",9.1256e6,9098223.2,"""Soles""",2018-07-17,2018-08-17,2018-09-03,"""Licitación Pública""","""Bien""","""LIMA""","""HOSPITAL CAYETANO HEREDIA""",31d,17d
2018,504515,"""CONTRATACI¿N DEL SERVICIO DE S…","""Contratado""",5.6953e6,5.6913e6,"""Soles""",2018-12-27,2019-01-15,2019-01-23,"""Adjudicación Simplificada""","""Servicio""","""CUSCO""","""UNIVERSIDAD NACIONAL DE SAN AN…",19d,8d
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2018,467305,"""EJECUCION DE LA OBRA: MEJORAMI…","""Contratado""",5.2887e6,5.2887e6,"""Soles""",2018-07-17,2018-09-26,2018-09-27,"""Licitación Pública""","""Obra""","""HUANUCO""","""MUNICIPALIDAD DISTRITAL DE YAC…",71d,1d
2018,493464,"""MEJORAMIENTO DE TROCHA CARROZA…","""Contratado""",1.3975e7,1.2577e7,"""Soles""",2018-11-15,2018-11-29,2018-12-04,"""Procedimiento Especial de Cont…","""Obra""","""LA LIBERTAD""","""MUNICIPALIDAD DISTRITAL DE CAC…",14d,5d
2018,494491,"""EJECUCION DE OBRA: MEJORAMIENT…","""Contratado""",1.6903e7,1.6903e7,"""Soles""",2018-10-05,2018-12-04,2018-12-06,"""Licitación Pública""","""Obra""","""UCAYALI""","""GOBIERNO REGIONAL DE UCAYALI S…",60d,2d


In [34]:
df_dias = df.with_columns([
    pl.col("f_convo_buenapro").dt.total_days().cast(pl.Int32),
    pl.col("f_buenapro_consentimiento").dt.total_days().cast(pl.Int32)
])


In [36]:
df_dias.write_csv("resultado.csv")